* Filter:
    - Importance: High => Low
    - Easy questions only

## Array.prototype.at()

* __USE THE `this` KEYWORD TO ACCESS THE OBJECT THAT YOU'RE CREATING THE PROTOTYPE FOR__
    - since we're creating a polyfill for the .at() method, we can use `this` to access the actual array to mutate it

In [ ]:
// my solution
/**
 * @param {number} index
 * @return {any | undefined}
 * 
 * arr = [42, 79]
 * arr[0] = 42
 * arr[1] = 79
 * arr[2] = undefined. if index >= arr.len
 * 
 * arr[-1] = 79
 * arr[-2] = 42
 * arr[-3] = undefined
 * 
 * arr.len + index => 2 + (-1) = 1 => 79
 * 2 + (-2) = 0 => 42
 * 2 + (-3) = -1 => undefined
 * if (arr.len + index < 0) return undefined
 * 
 * so index is guaranteed to be a number
 * it can be a floaty number so must convert to an integer
 */
Array.prototype.myAt = function (index) {
  // if index is not an actual integer, we must convert
  index = Math.trunc(index);
  
  const { length } = this;
  if (index >= length) return;

  if (index < 0) {
    // negative index
    index = length + index;
  }

  if (index < 0) return;

  return this[index];
};

In [ ]:
// actual solution

/**
 * @param {number} index
 * @return {any | undefined}
 */
Array.prototype.myAt = function (index) {
  const len = this.length;
  if (index < -len || index >= len) {
    return;
  }

  return this[(index + len) % len];
};


## Array.prototype.filter()

* under the hood, Arrays are basically objects
```
{
    0: "first_item",
    1: "second_item",
    2: "third_item",
    length: 3
}
["first_item", "second_item", "third_item"]
```
* to check if arr[i] is missing for sparse arrays, e.g. [1, ,3, 4], you would use `if (i in this)`
    - this is like checking `if (key in obj)` but in this case, it's seeing if arr[i] is undefined
* thisArg, which is the 2nd argument accepted by myFilter, refers to the `this` object in the context of the callbackFn
    - so if the callbackFn uses this.[value], that `this` refers to thisArg
```
callbackFn = function(value, index) {
    // the this for this.label refers to thisArg
    console.log(`${this.label}: ${value} + ${index}`);
}
thisArg = { label: "number" };
[1, 2].forEach(callbackFn, thisArg); // prints out "number: 1 + 0"
```
* __keep in mind, if your callbackFn is an `arrow function: () => {}`, passing in an argument for thisArg would NOT work because `this` for arrow functions is lexically scoped to whatever is calling it, so you can't change it in .call(), .apply(), or .bind()__
    - __ARROW FUNCTIONS DO NOT HAVE THEIR OWN `this`__
    - if you used a normal method: `function(){}`, then you can pass in an argument for thisArg to set `this` as thisArg using .call(), .apply(), .bind()
    - __NORMAL METHODS LOSE THEIR `this` because there's nothing before the dot (".").__
    - __IF YOU CALLED `obj.normalMethod()`, THEN `this` REFERS TO `obj`__
    - __BUT WHEN YOU USE IT AS A CALLBACK FUNCTION (e.g. like `.forEach` or `.Map` or in event handlers) YOU BASICALLY JUST CALL IT LIKE A NORMAL FUNCTION, NOT A METHOD CALL, i.e. `normalMethod` WITHOUT ANYTHING BEFORE IT__

In [ ]:
/**
 * @template T
 * @param { (value: T, index: number, array: Array<T>) => boolean } callbackFn
 * @param {any} [thisArg]
 * @return {Array<T>}
 * 
 * if result of callbackFn => true, push result of callback into the newArr
 * for sparse arrays, we have to check if there is an item at arr[i]
 * we can do this with if (i in this)
 */
Array.prototype.myFilter = function (callbackFn, thisArg) {
  const newArr = [];
  const len = this.length;

  for (let i = 0; i < len; i++) {
    // check if i is even in the array
    if (i in this) {
      // call the callbackfunction
      // thisArg = refers to the this that the callbackfunction is calling
      // inside of it
      // then the rest of the arguments match to what is accepted by filter
      // so it's the item, index, and original array itself
      const isMatch = callbackFn.call(thisArg, this[i], i, this);
      if (isMatch) {
        newArr.push(this[i]);
      } 
    }
  }
  return newArr;
};

## Array.prototype.map()

* pretty much the same as filter
* although I did mess up .call() with .apply()
    - `Function.prototype.call(thisArg, arg1, arg2, arg3,..., argn)`
    - `Function.prototype.apply(thisArg, [arg1, arg2, arg3, ..., argn]`

In [ ]:
/**
 * @template T, U
 * @param { (value: T, index: number, array: Array<T>) => U } callbackFn
 * @param {any} [thisArg]
 * @return {Array<U>}
 * 
 * for each element in the array, we call a callback function on it
 * 
 * what happens if we have a sparse array?
 *  - the new array will still be sparse
 *  - [0, , 2].map((val) => val * 2) = [0, , 4];
 *  - how do we mimic this behavior?
 *  - Array.from({ length: originalArrLen })
 *  - this ensures that we allocate enough space and can safely skip over it
 *  - but still keep the sparse index
 */
Array.prototype.myMap = function (callbackFn, thisArg) {
  const len = this.length;
  const newArr = Array.from({ length: len });

  for (let i = 0; i < len; i++) {
    // if arr[i] has a value, we can call the callbackFn on it
    if (i in this) {
      // map((value, index, array))
      newArr[i] = callbackFn.call(thisArg, this[i], i, this);
    }
  }

  return newArr;
};